# Import libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.models import densenet161, DenseNet161_Weights
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
from PIL import Image
import os
import numpy as np

# Database creations using pytorch Dataset 

In [2]:
class ImageAuthenticityDataset(Dataset):
    """Dataset for image quality assessment."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the CSV file with annotations.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.dir_path = os.path.dirname(csv_file)  # Directory of the CSV file

    def __len__(self):
        """Returns the number of samples in the dataset."""
        return len(self.data)

    def __getitem__(self, idx,):
        """
        Retrieves an image and its labels by index.

        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            tuple: A tuple (image, labels) where:
                image (PIL.Image): The image.
                labels (torch.Tensor): Tensor containing quality and authenticity scores.
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # TODO: to be fixed, right now is folder dependent
        img_name = self.data.iloc[idx, 3].replace("./", "../../")
        image = Image.open(img_name).convert('RGB')
        authenticity = self.data.iloc[idx, 1]  # Authenticity column
        labels = torch.tensor([authenticity], dtype=torch.float)


        if self.transform:
            image = self.transform(image)

        return image, labels


# Definitions of the models

In [3]:
class AuthenticityPredictor(nn.Module):
    def __init__(self, freeze_backbone=True):
        super().__init__()
        # Load pre-trained DenseNet-161
        densenet = densenet161(weights=DenseNet161_Weights.DEFAULT)
        
        # Freeze backbone if requested
        if freeze_backbone:
            for param in densenet.parameters():
                param.requires_grad = False
                
        # Store the features
        self.features = densenet.features
        
        # DenseNet already includes a ReLU and pooling after features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        # DenseNet-161's output feature dimension is 2208 instead of 2048
        self.regression_head = nn.Sequential(
                nn.Linear(2208, 512),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(512, 128),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(128, 1)
            )
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        predictions = self.regression_head(x)
        return predictions, x  # Return predictions and features

# Training and evaluation functions

In [4]:

def test_model(model, dataloader, criterion, device='cuda'):

    """
    Tests the model on the test dataset.

    Args:
        model (nn.Module): The trained model.
        dataloader (DataLoader): The test data loader.
        criterion (nn.Module): The loss function.
        device (str): Device to use for testing ('cuda' or 'cpu'). Defaults to 'cuda'.

    Returns:
        float: The average loss on the test dataset.
    """
    model.eval()  # Set the model to evaluation mode
    model.to(device)
    running_loss = 0.0

    with torch.no_grad():  # Disable gradient calculation
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs, _ = model(inputs)
            
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)

    mse_test_loss = running_loss / len(dataloader.dataset)
    rmse_test_loss = np.sqrt(mse_test_loss)

    test_loss = rmse_test_loss  # or mse_test_loss, depending on your preference

    return test_loss


# Pruning funcitons

In [5]:
from tqdm import tqdm

def compute_feature_map_importance(model, dataloader, device, layer_name) -> tuple[np.ndarray, np.ndarray]:
    """Computes the importance of each feature map in a convolution
    layer by measuring the change in predictions when the feature map is zero
    out.

    Returns:
        tuple: (indices, importance_scores) where both are numpy arrays
    """
    #if importance_scores.npy exists, load it
    if os.path.exists('Ranking_arrays/real_authenticity_batch_importance_scores.npy'):
        print("Importance scores already computed, loading from file")
        return np.load('Ranking_arrays/real_authenticity_batch_importance_scores.npy')

    model.eval()
    model.to(device)
    importance_scores = []
    dict_modules = dict(model.named_modules())
    layer = dict_modules[layer_name]
    baseline_authenticity_rmse = test_model(model, dataloader, nn.MSELoss(), device=device)


    print(f'Authenticity baseline RMSE: {baseline_authenticity_rmse:.4f}')

    with torch.no_grad():
        for i in tqdm(range(layer.out_channels), desc=f"Computing importance for {layer_name}"):
            # Create a backup of the weights and bias
            backup_weights = layer.weight[i, ...].clone()
            backup_bias = layer.bias[i].clone() if layer.bias is not None else None

            # Zero out the i-th output channel
            layer.weight[i, ...] = 0
            if layer.bias is not None:
                layer.bias[i] = 0


            pruned_authenticity_rmse = test_model(model, dataloader, nn.MSELoss(), device=device)


            # Compute importance score
            importance_score = baseline_authenticity_rmse - pruned_authenticity_rmse
            importance_scores.append([i, importance_score])

            # After computing importance, restore weights and bias
            layer.weight[i, ...] = backup_weights
            if layer.bias is not None:
                layer.bias[i] = backup_bias

    sorted_importance_scores = sorted(importance_scores, key=lambda x: x[1], reverse=True)
    # save np array
    np.save('Ranking_arrays/real_authenticity_batch_importance_scores.np', sorted_importance_scores)
    return np.array(sorted_importance_scores)

def remove_noisy_feature_maps(model, dataloader, device, layer_name, sorted_importance_scores, model_path='Weights/pruned_model.pth'):
    """
    Remove noisy feature maps from a convolutional layer based on importance scores.
    Feature maps are zeroed out one by one and kept zeroed only if model performance improves.

    Args:
        model: The neural network model
        dataloader: DataLoader for evaluation
        device: Device to run the model on (cuda/cpu)
        layer_name: Name of the layer to optimize
        sorted_importance_scores: List of tuples (channel_index, importance_score) sorted by importance

    Returns:
        Dictionary with pruning results and performance metrics
    """
    model.eval()
    model.to(device)

    # Get the target layer
    dict_modules = dict(model.named_modules())
    layer = dict_modules[layer_name]

    # Create a backup of the original weights and bias
    original_weights = layer.weight.clone()
    original_bias = layer.bias.clone() if layer.bias is not None else None

    # Initialize tracking variables
    removed_features = []
    rmse_history = []

    # Get baseline performance
    baseline_authenticity_rmse = test_model(model, dataloader, nn.MSELoss(), device=device)


    print(f"Baseline authenticity RMSE: {baseline_authenticity_rmse:.4f}")
    print("------------------")

    # Track initial performance
    rmse_history.append(([], baseline_authenticity_rmse))
    baseline_rmse = baseline_authenticity_rmse

    # Iterate over the sorted indices and if removing a feature map improves performance, keep it removed
    for idx, (channel_idx, importance_score) in tqdm(enumerate(sorted_importance_scores), total=len(sorted_importance_scores), desc=f"Removing noisy features from {layer_name}"):
        channel_idx = int(channel_idx)

        # Temporarily zero out this feature map
        layer.weight[channel_idx, ...] = 0
        if layer.bias is not None:
            layer.bias[channel_idx] = 0

        # Evaluate model with feature map removed

        authenticity_pruned_rmse = test_model(model, dataloader, nn.MSELoss(), device=device)

        print(f"Iteration {idx+1}/{len(sorted_importance_scores)}: " +
              f"Testing removal of channel {channel_idx}, " +
              f"Importance: {importance_score:.4f}, " +
              f"RMSE: {authenticity_pruned_rmse:.4f}")

        # Decide whether to keep this feature map removed
        if authenticity_pruned_rmse < baseline_rmse:
            baseline_rmse = authenticity_pruned_rmse # Update baseline RMSE
            removed_features.append(channel_idx)
            rmse_history.append((removed_features.copy(), baseline_rmse))
            print(f"  ✓ IMPROVING: Zeroing out feature map {channel_idx}")
        else:
            # Restore the feature map
            layer.weight[channel_idx, ...] = original_weights[channel_idx, ...]
            if layer.bias is not None:
                layer.bias[channel_idx] = original_bias[channel_idx]
            print(f"  ✗ NOT IMPROVING: Keeping feature map {channel_idx}")

        print(f"  Current best RMSE: {baseline_rmse:.4f}")
        print("------------------")

    # Final statistics
    print("\n------------------")
    print(f"Final RMSE: {baseline_rmse:.4f} after removing {len(removed_features)} feature maps")
    print(f"Improvement over baseline: {baseline_authenticity_rmse - baseline_rmse:.4f}")
    print(f"Feature reduction: {(len(removed_features)/len(sorted_importance_scores))*100:.1f}%")

    # Save the pruned model
    torch.save(model.state_dict(), model_path)

    return {
        'removed_features': removed_features,
        'baseline_rmse': baseline_authenticity_rmse,
        'final_rmse': baseline_rmse,
        'improvement': baseline_authenticity_rmse - baseline_rmse,
        'reduction_percentage': (len(removed_features)/len(sorted_importance_scores))*100,
        'rmse_history': rmse_history
    }

def remove_negative_impact_feature_maps(model, dataloader, device, layer_name, sorted_importance_scores, model_path='Weights/negative_impact_pruned_model.pth'):
    """
    Remove feature maps that have a negative impact on model performance based on importance scores (impotance score > 0).

    Args:
        model: The neural network model
        dataloader: DataLoader for evaluation
        device: Device to run the model on (cuda/cpu)
        layer_name: Name of the layer to optimize
        sorted_importance_scores: List of tuples (channel_index, importance_score) sorted by importance

    Returns:
        Dictionary with pruning results and performance metrics
    """
    model.eval()
    model.to(device)

    # Get the target layer
    dict_modules = dict(model.named_modules())
    layer = dict_modules[layer_name]

    # Create a backup of the original weights and bias
    original_weights = layer.weight.clone()
    original_bias = layer.bias.clone() if layer.bias is not None else None

    # Get baseline performance

    authenticity_rmse = test_model(model, dataloader, nn.MSELoss(), device=device)

    # Initialize tracking variables
    removed_features = []

    # Iterate over the sorted indices and zero out all the feature maps that have a negative impact (importance < 0)

    for idx, (channel_idx, importance_score) in tqdm(enumerate(sorted_importance_scores), total=len(sorted_importance_scores), desc=f"Removing negative impact features from {layer_name}"):
        print(f"Iteration {idx} - Channel {channel_idx}: Importance score: {importance_score:.4f}")
        if importance_score > 0:
            channel_idx = int(channel_idx)
            layer.weight[channel_idx, ...] = 0
            if layer.bias is not None:
                layer.bias[channel_idx] = 0
            removed_features.append(channel_idx)

    # Evaluate model with feature maps removed
    new_authenticity_rmse = test_model(model, dataloader, nn.MSELoss(), device=device)

    # Save the pruned model
    torch.save(model.state_dict(), model_path)

    # Restore original weights for future use
    layer.weight.data.copy_(original_weights)
    if layer.bias is not None:
        layer.bias.data.copy_(original_bias)


    return {
        'removed_features': removed_features,
        'baseline_rmse': authenticity_rmse,
        'final_rmse': new_authenticity_rmse,
        'improvement': authenticity_rmse - new_authenticity_rmse,
        'reduction_percentage': (len(removed_features)/len(sorted_importance_scores))*100
    }


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data transformations for the ImageNet dataset
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

annotations_file = '../../Dataset/AIGCIQA2023/real_images_annotations.csv'

# Create the dataset
dataset = ImageAuthenticityDataset(csv_file=annotations_file, transform=data_transforms)

# Set random seeds for reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


# Create data loaders
BATCH_SIZE = 64 # Set to 1 for handling individual images
EPOCHS = 20

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)


# Create a dictionary containing the data loaders
dataloaders = {
    'train': train_dataloader,
    'val': val_dataloader,
    'test': test_dataloader
}

model = AuthenticityPredictor()
criterion = nn.MSELoss()  # Mean Squared Error Loss (regression)
optimizer = optim.Adam(model.regression_head.parameters(), lr=0.001)

In [7]:
# LAYER to prune
LAYER = 'features.denseblock4.denselayer24.conv2'
DEVICE = 'cuda'

# Base model for importance score computation
base_model = AuthenticityPredictor()
base_model.load_state_dict(torch.load('Weights/DenseNet-161_real_authenticity_finetuned.pth'))
base_model.eval()
base_model.to(DEVICE)

sorted_importance_scores = compute_feature_map_importance(base_model, train_dataloader, DEVICE, LAYER)
np.save('Ranking_arrays/real_authenticity_importance_scores.npy', sorted_importance_scores)
del base_model

# Model for negative impact feature maps removal
negative_impact_model = AuthenticityPredictor()
negative_impact_model.load_state_dict(torch.load('Weights/DenseNet-161_real_authenticity_finetuned.pth'))
negative_impact_model.eval()
negative_impact_model.to(DEVICE)

negative_impact_subset = remove_negative_impact_feature_maps(negative_impact_model, train_dataloader, DEVICE, LAYER, sorted_importance_scores, model_path='Weights/real_authenticity_negative_impact_pruned_model.pth')
print(negative_impact_subset)

del negative_impact_model


# Model for noisy feature maps removal
noisy_pruning_model = AuthenticityPredictor()
noisy_pruning_model.load_state_dict(torch.load('Weights/DenseNet-161_real_authenticity_finetuned.pth'))
noisy_pruning_model.eval()
noisy_pruning_model.to(DEVICE)

noisy_optimal_subset = remove_noisy_feature_maps(noisy_pruning_model, train_dataloader, DEVICE, LAYER, sorted_importance_scores, model_path='Weights/real_authenticity_noise_out_pruned_model.pth')

del noisy_pruning_model



/tmp/ipykernel_3786844/3782981658.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  base_model.load_state_dict(torch.load('Weights/DenseNet-161_real_authenticity_finetuned

Authenticity baseline RMSE: 4.4782


Computing importance for features.denseblock4.denselayer24.conv2: 100%|██████████| 48/48 [32:01<00:00, 40.03s/it]
/tmp/ipykernel_3786844/3782981658.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to t

Iteration 0 - Channel 7.0: Importance score: 0.1125
Iteration 1 - Channel 32.0: Importance score: 0.0933
Iteration 2 - Channel 29.0: Importance score: 0.0555
Iteration 3 - Channel 11.0: Importance score: 0.0544
Iteration 4 - Channel 6.0: Importance score: 0.0459
Iteration 5 - Channel 17.0: Importance score: 0.0411
Iteration 6 - Channel 21.0: Importance score: 0.0404
Iteration 7 - Channel 14.0: Importance score: 0.0384
Iteration 8 - Channel 13.0: Importance score: 0.0277
Iteration 9 - Channel 47.0: Importance score: 0.0268
Iteration 10 - Channel 3.0: Importance score: 0.0188
Iteration 11 - Channel 12.0: Importance score: 0.0187
Iteration 12 - Channel 23.0: Importance score: 0.0129
Iteration 13 - Channel 24.0: Importance score: 0.0097
Iteration 14 - Channel 25.0: Importance score: 0.0061
Iteration 15 - Channel 36.0: Importance score: 0.0046
Iteration 16 - Channel 18.0: Importance score: 0.0040
Iteration 17 - Channel 31.0: Importance score: 0.0034
Iteration 18 - Channel 19.0: Importance s

/tmp/ipykernel_3786844/3782981658.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  noisy_pruning_model.load_state_dict(torch.load('Weights/DenseNet-161_real_authenticity

...

Iteration 46/48: Testing removal of channel 4, Importance: -0.1394, RMSE: 4.1297
  ✗ NOT IMPROVING: Keeping feature map 4
  Current best RMSE: 4.0603
------------------


Removing noisy features from features.denseblock4.denselayer24.conv2:  98%|█████████▊| 47/48 [31:13<00:41, 41.78s/it]

Iteration 47/48: Testing removal of channel 44, Importance: -0.1818, RMSE: 4.1609
  ✗ NOT IMPROVING: Keeping feature map 44
  Current best RMSE: 4.0603
------------------


Removing noisy features from features.denseblock4.denselayer24.conv2: 100%|██████████| 48/48 [31:53<00:00, 39.86s/it]

Iteration 48/48: Testing removal of channel 40, Importance: -0.2098, RMSE: 4.1562
  ✗ NOT IMPROVING: Keeping feature map 40
  Current best RMSE: 4.0603
------------------

------------------
Final RMSE: 4.0603 after removing 17 feature maps
Improvement over baseline: 0.4180
Feature reduction: 35.4%
